```
pip install apio
```

On Windows if you get a permission error:

```
(python.exe -m pip install apio)
```

Once `apio` is installed, installed the `oss-cad-suite`:

```
apio install oss-cad-suite
```

Other links:
https://github.com/YosysHQ/nextpnr/issues/660
https://github.com/tgingold/OpenTDC/tree/master

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_NEORV32'
SS_VER = 'SS_VER_1_0'

In [7]:
%run "c:/ChipWhisperer570_64/cw/home/portable/chipwhisperer/jupyter/Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [10]:
scope.io.tio1 = None
scope.io.tio2 = None

tio1            = high_z
tio2            = high_z
tio3            = high_z
tio4            = high_z
pdid            = low
pdic            = high_z
nrst            = high
glitch_hp       = False
glitch_lp       = False
extclk_src      = hs1
hs2             = clkgen
target_pwr      = True
tio_states      = (1, 1, 1, 0)
cdc_settings    = bytearray(b'\x00\x00\x00\x00')
aux_io_mcx      = high_z
glitch_trig_mcx = trigger

In [53]:
import subprocess
subprocess.run("apio build", cwd="rtl-delaytests", shell=True, check=True)

CompletedProcess(args='apio build', returncode=0)

In [54]:
from chipwhisperer.hardware.naeusb.programmer_targetfpga import LatticeICE40
bs = r"C:\Users\cofly\Dropbox (NewAE Technology Inc)\engineering\research\ice40-glitch-meter\cw308_real\rtl-delaytests\hardware.bin"

ice = LatticeICE40(scope)

def reboot_flush():
    scope.clock.clkgen_freq = 7.37E6
    
    ice.erase_and_init()
    ice.program(bs)

    scope.clock.clkgen_freq = 25E6

reboot_flush()

Measured using picoscope with ETS (10 GS/s):

For carry-line implementation:
1.1V:
1: 2.692 nS
2: 3.425 nS
3: 3.660 nS
4: 4.260 nS

1.2V:
1: 2.218 nS
2: 2.758 nS
3: 2.905 nS
4: 3.312 nS


1.3V: 
1: 1.890 nS
2: 2.320 nS
3: 2.460 nS
4: 2.792 nS

For LUT implementation:
1.1V:
1: 2.134 nS
2: 3.770 nS
3: 5.172 nS
4: 8.408 nS

1.2V:
1: 1.767 nS
2: 2.990 nS
3: 3.935 nS
4: 6.113 nS

1.3V:
1: 1.536 nS
2: 2.516 nS
3: 3.218 nS
4: 4.909 nS

In [104]:
datastr_tdc = """1.1V:
1: 2.692 nS
2: 3.425 nS
3: 3.660 nS
4: 4.260 nS

1.2V:
1: 2.218 nS
2: 2.758 nS
3: 2.905 nS
4: 3.312 nS

1.3V:
1: 1.890 nS
2: 2.320 nS
3: 2.460 nS
4: 2.792 nS"""

datastr_lut = """1.1V:
1: 2.134 nS
2: 3.770 nS
3: 5.172 nS
4: 8.408 nS

1.2V:
1: 1.767 nS
2: 2.990 nS
3: 3.935 nS
4: 6.113 nS

1.3V:
1: 1.536 nS
2: 2.516 nS
3: 3.218 nS
4: 4.909 nS"""

In [106]:
import numpy as np

def print_stats(datastr):
    starts = [datastr.find("1.1V:"), datastr.find("1.2V:"), datastr.find("1.3V")]
    data1v1 = datastr[starts[0]:starts[1]][6:]
    data1v2 = datastr[starts[1]:starts[2]][6:]
    data1v3 = datastr[starts[2]:][6:]

    datasets = [data1v1, data1v2, data1v3]
    for data in datasets:
        datasplit = data.split('\n')[0:4]
        times = []
        for point in datasplit:
            items = point.split()
            times.append(float(items[1]))

        diffs = [times[1]-times[0], times[2]-times[1], times[3]-times[2]]

        print("mean = %f, stddev = %f"%(np.mean(diffs), np.std(diffs)))

print("Using carry chain:")
print_stats(datastr_tdc)

print("Using LUT:")
print_stats(datastr_lut)

Using carry chain:
mean = 0.522667, stddev = 0.210533
mean = 0.364667, stddev = 0.163210
mean = 0.300667, stddev = 0.120447
Using LUT:
mean = 2.091333, stddev = 0.815020
mean = 1.448667, stddev = 0.528057
mean = 1.124333, stddev = 0.416457
